In [ ]:
#@title Run this to import random bios { display-mode: "form" }

import requests
import pandas as pd
import time
import random
import re
import numpy as np
import _pickle as pickle
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup as bs


## Using BeautifulSoup

# Randomizing the refresh rate
seq = [i/10 for i in range(8,18)]

# Creating a list of bios
biolist = []

# Gathering bios by looping and refreshing the web page
for _ in tqdm(range(100)):
    
    # Refreshing the page
    page = requests.get("https://www.fakepersongenerator.com/user-biography-generator?new=refresh")
    soup = bs(page.content)
    
    try:
        # Getting the bios
        bios = soup.find('div', class_='row no-margin for-sign').find_all('p')

        # Adding to a list of the bios
        biolist.extend([re.findall('"([^"]*)"', i.text) for i in bios])
    except:
        pass
    
    # Sleeping 
    time.sleep(random.choice(seq))
    
# Creating a DF from the bio list
bio_df = pd.DataFrame(biolist, columns=['Bios'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
#@title More stuff on adding topics to our profiles (1-10 for how much you are interested in each topic) { display-mode: "form" }
length=len(bio_df)
ageList=[]
genderList=[]
topics=['Movies', 'Video Games', 'Politics']
topicsList=[]
for i in range(length):
  ageList.append(random.randint(13, 25))
  genderList.append(random.randint(0, 1))
  topicsList.append([random.randint(1, 10), 
                     random.randint(1, 10), random.randint(1, 10)])

age_df = pd.DataFrame(ageList, columns=['Age'])
gender_df = pd.DataFrame(genderList, columns=['Gender'])
topics_df = pd.DataFrame(topicsList, columns=topics)

df=bio_df.join(age_df).join(gender_df).join(topics_df)



In [ ]:
print(df.head().to_markdown())

|    | Bios                                                                                                                                                  |   Age |   Gender |   Movies |   Video Games |   Politics |
|---:|:------------------------------------------------------------------------------------------------------------------------------------------------------|------:|---------:|---------:|--------------:|-----------:|
|  0 | Passionate student. Freelance web geek. Amateur music enthusiast. Incurable social media scholar.                                                     |    19 |        0 |       10 |             6 |          2 |
|  1 | Problem solver. Social media trailblazer. Wannabe travelaholic. Subtly charming analyst. Coffee ninja.                                                |    24 |        1 |        8 |             4 |          3 |
|  2 | Extreme bacon buff. Organizer. Evil pop culture practitioner. Beer nerd. Writer. Lifelong web scholar. Freelance zombie f

In [ ]:
#@title teach the computer english
import re
import gdown
import seaborn as sns
import pandas as pd
import numpy as np
from torchtext.vocab import GloVe
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
import requests, io, zipfile

In [ ]:
VEC_SIZE = 300
glove = GloVe(name='6B', dim=VEC_SIZE)

# Returns word vector for word if it exists, else return None.
def get_word_vector(word):
    try:
      return glove.vectors[glove.stoi[word.lower()]].numpy()
    except KeyError:
      return None

.vector_cache/glove.6B.zip: 862MB [02:42, 5.30MB/s]                           
100%|█████████▉| 399072/400000 [00:42<00:00, 9463.94it/s]

In [ ]:
#@title testing the cool word simularity machine { run: "auto", display-mode: "both" }

def cosine_similarity(vec1, vec2):    
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

word1 = "ethnicity" #@param {type:"string"}
word2 = "potatoes" #@param {type:"string"}

print('Word 1:', word1)
print('Word 2:', word2)

def cosine_similarity_of_words(word1, word2):
  vec1 = get_word_vector(word1)
  vec2 = get_word_vector(word2)
  
  if vec1 is None:
    print(word1, 'is not a valid word. Try another.')
  if vec2 is None:
    print(word2, 'is not a valid word. Try another.')
  if vec1 is None or vec2 is None:
    return None
  
  return cosine_similarity(vec1, vec2)
  

print('\nCosine similarity:', cosine_similarity_of_words(word1, word2))

Word 1: ethnicity
Word 2: potatoes

Cosine similarity: 0.01731634
INPUT: I am a 15-yrs old asian teen, play minecraft, and love biking
OUTPUT: I am a 15-yrs old !$%^@!$!^% teen, play minecraft, and love biking


In [ ]:
def getBioVector(bio):
  bio=bio.strip()
  found_words=0.0
  X=np.zeros(VEC_SIZE)
  for word in bio.split():  
    vec = get_word_vector(word)
    if vec is not None:
        found_words += 1
        X += vec
  if found_words > 0:
        X /= found_words
  return X
def getBioScore(bio1, bio2):
  return cosine_similarity(getBioVector(bio1), getBioVector(bio2))

bio1 = "I play minecarft and click quickly on kids" #@param {type:"string"}
bio2 = "I am such an environmentalist" #@param {type:"string"}

print('Bio 1:', bio1)
print('Bio 2:', bio2)
print('\nCosine similarity:', getBioScore(bio1, bio2))


In [ ]:

def bestMatches(person, lim):
  matches=[]
  for i in range(len(df)):
    if(i!=person):
      if(abs(df['Age'][i]-df['Age'][person]) <= 2 and df['Gender'][i]!=df['Gender'][person]):
        matches.append([getBioScore(df['Bios'][i], df['Bios'][person]), i] )
  matches.sort(reverse=True)
  for i in range(min(lim, len(matches))):
    print(matches[i][1], df['Bios'][matches[i][1]])

print("THIS GUY MATCHES WITH:", df['Bios'][0])

bestMatches(0, 5)

THIS GUY MATCHES WITH: Beer specialist. Wannabe alcoholaholic. Prone to fits of apathy. Infuriatingly humble twitter evangelist. Evil writer. Tv geek.
479 Typical beer junkie. Tv fan. Prone to fits of apathy. Twitter ninja. Web fanatic. Amateur writer.
326 Food evangelist. Organizer. Unapologetic music buff. Prone to fits of apathy. Infuriatingly humble tvaholic.
453 Proud student. Writer. Web practitioner. Thinker. Friendly beer guru. Infuriatingly humble twitter enthusiast. Social media lover.
548 Twitter specialist. Extreme entrepreneur. Communicator. Evil student. Professional beer junkie.
261 Professional pop culture guru. Lifelong tv fan. Proud travel fanatic. Prone to fits of apathy.
